### Initially created a solution using pandas and other libraries to prototype the approach. Once the solution was working, we re-implemented it using Spark. 
##### Tested also the concept of batches and its performance optimizations.  

In [4]:
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm
import numpy as np

def process_batch(batch_df):
    """Process a single batch of data"""
    # Extract salary range into min and max
    def extract_salary_range(salary_str):
        try:
            salary_str = str(salary_str).replace('$', '').replace('£', '').replace('€', '')
            parts = salary_str.replace(',', '').split('-')
            if len(parts) == 2:
                return pd.Series([float(parts[0].strip().replace('K', '000')), 
                                float(parts[1].strip().replace('K', '000'))])
            else:
                return pd.Series([None, None])
        except:
            return pd.Series([None, None])
    
    # Process salary ranges
    batch_df[['min_salary', 'max_salary']] = batch_df['Salary Range'].apply(extract_salary_range)
    
    # Create new dataframe with selected columns
    new_batch = pd.DataFrame({
        'job_description': batch_df['Job Description'],
        'job_skills': batch_df['skills'],
        'job_industry': batch_df['Company Profile'],
        'job_location': batch_df['location'] + ', ' + batch_df['Country'],
        'experience_level': batch_df['Experience'],
        'min_salary': batch_df['min_salary'],
        'max_salary': batch_df['max_salary'],
        'job_posting_date': pd.to_datetime(batch_df['Job Posting Date']).dt.date
    })
    
    # Clean up missing values
    new_batch = new_batch.replace('', pd.NA)
    new_batch = new_batch.replace('NaN', pd.NA)
    
    return new_batch

def process_csv(input_file, output_file, batch_size=1000):
    """
    Process CSV file in batches to select and rename specific columns.
    
    Parameters:
    input_file (str): Path to input CSV file
    output_file (str): Path to output CSV file
    batch_size (int): Number of rows to process in each batch
    """
    print(f"\n Starting CSV processing at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f" Reading file: {input_file}")
    
    try:
        # Count total rows first
        total_rows = sum(1 for _ in open(input_file)) - 1  # subtract header row
        print(f" Total rows to process: {total_rows:,}")
        
        # Process in batches
        processed_dfs = []
        batch_count = 0
        
        # Create a chunked iterator
        chunks = pd.read_csv(input_file, chunksize=batch_size)
        
        # Calculate number of batches for progress bar
        n_batches = int(np.ceil(total_rows / batch_size))
        
        print("\n Processing batches:")
        # Process each batch with progress bar
        for chunk in tqdm(chunks, total=n_batches, desc="Processing"):
            processed_batch = process_batch(chunk)
            processed_dfs.append(processed_batch)
            batch_count += 1
            
            # Print batch statistics every 5 batches
            if batch_count % 5 == 0:
                print(f"\n Processed {batch_count * batch_size:,} rows...")
        
        print("\n Combining all batches...")
        final_df = pd.concat(processed_dfs, ignore_index=True)
        
        # Print final statistics
        print("\n Processing Statistics:")
        print(f"Total records processed: {len(final_df):,}")
        print(f"Records with valid salary range: {final_df['min_salary'].notna().sum():,}")
        print(f"Unique locations: {final_df['job_location'].nunique():,}")
        print(f"Date range: {final_df['job_posting_date'].min()} to {final_df['job_posting_date'].max()}")
        
        print(f"\n Saving processed data to {output_file}")
        # Save to new CSV file
        final_df.to_csv(output_file, index=False)
        
        print(f" Processing completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        return final_df
        
    except Exception as e:
        print(f" Error processing CSV file: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    input_file = "/home/ubuntu/job_descriptions.csv"  # Replace with your input file path
    output_file = "/home/ubuntu/job_descriptions_modified.csv"   # Replace with your desired output file path
    
    # You can adjust the batch_size based on your system's memory capacity
    processed_df = process_csv(input_file, output_file, batch_size=1000)


🚀 Starting CSV processing at 2024-11-24 10:35:25
📂 Reading file: /home/ubuntu/job_descriptions.csv
📊 Total rows to process: 1,615,940

⏳ Processing batches:


Processing:   0%|          | 8/1616 [00:00<01:33, 17.25it/s]


📈 Processed 5,000 rows...


Processing:   1%|          | 12/1616 [00:00<01:35, 16.76it/s]


📈 Processed 10,000 rows...


...


📈 Processed 1,605,000 rows...


Processing: 100%|█████████▉| 1612/1616 [01:51<00:00, 15.42it/s]


📈 Processed 1,610,000 rows...


Processing: 100%|██████████| 1616/1616 [01:51<00:00, 14.46it/s]



📈 Processed 1,615,000 rows...

🔄 Combining all batches...

📊 Processing Statistics:
Total records processed: 1,615,940
Records with valid salary range: 1,615,940
Unique locations: 216
Date range: 2021-09-15 to 2023-09-15

💾 Saving processed data to /home/ubuntu/job_descriptions_modified.csv
✅ Processing completed at 2024-11-24 10:37:45
